In [25]:
import requests
from bs4 import BeautifulSoup
from time import sleep
import pandas as pd

In [26]:
# Главный URL для скрэпинга и URL каталога марок и моделей
# Main URL for scraping and URL of marks and models catalog
URL = 'https://auto.ru/-/ajax/desktop/listing/'
URL_CATALOG = 'https://auto.ru/htmlsitemap/mark_model_catalog.html'

# Заголовки веб-страницы
# Webpage headers
HEADERS = {
    'Accept': '*/*',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7',
    'Connection': 'keep-alive',
    'Content-Length': '77',
    'content-type': 'application/json',
    'Cookie': '_csrf_token=23608122d7607cf614fa3696f170304f2c942825086d7100; autoru_sid=a%3Ag5e54cb202kbrp94k5stjgeuok5qc0lr.a4bf836f5de7a860384d4bb97a55d0e0%7C1582615328026.604800.Q6xoprzzbkFU9J433uQN0A.34nVWWhPYMf2gQUyYdxm-hAPmkz-UZuRfIKcHxDth7w; autoruuid=g5e54cb202kbrp94k5stjgeuok5qc0lr.a4bf836f5de7a860384d4bb97a55d0e0; suid=f70cb3a665b30dabcc2aefb60beba688.092164db5081dbf54d3645dd1160a738; yandexuid=4412548211582615329; _ym_wasSynced=%7B%22time%22%3A1582615332141%2C%22params%22%3A%7B%22eu%22%3A0%7D%2C%22bkParams%22%3A%7B%7D%7D; gdpr=0; _ym_uid=1582615332878294126; _ym_isad=2; gids=; from=direct; X-Vertis-DC=myt; _ym_visorc_22753222=b; cycada=1Ra6P74hpA1DJTHrQBDvqC2RWMZ0zfqbWbqR95lvoFc=; from_lifetime=1582636077325; _ym_d=1582636077',
    'Host': 'auto.ru',
    'Origin': 'https://auto.ru',
    'Referer': 'https://auto.ru/cars/used/?sort=fresh_relevance_1-desc&page=1',
    'Sec-Fetch-Dest': 'empty',
    'Sec-Fetch-Mode': 'same-origin',
    'Sec-Fetch-Site': 'same-origin',
    'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 \
    (KHTML, like Gecko) Chrome/80.0.3987.116 Safari/537.36',
    'x-client-app-version': '202002.20.164944',
    'x-client-date': '1582636185640',
    'x-csrf-token': '23608122d7607cf614fa3696f170304f2c942825086d7100',
    'x-page-request-id': '4e70327218c7ba488485edc641875179',
    'x-requested-with': 'fetch'
}

# Параметры запроса
# Request parameters
PARAMS = {
    'section': 'used',
    'category': 'cars',
    'sort': 'fresh_relevance_1-desc',
    'geo_radius': 500,
    'geo_id': 213,
    "top_days": 0
}

# Шаблон считываемых характеристик автомобиля
# Pattern of vehicle characteristics to read
CAR_PATTERN = {
    "bodyType"            : "['vehicle_info']['configuration']['human_name']",
    "brand"               : "['vehicle_info']['mark_info']['code']",
    "model_name"          : "['vehicle_info']['model_info']['code']",
    "color"               : "['color_hex']",
    "fuelType"            : "['lk_summary'].split()[-1]",
    "modelDate"           : "['vehicle_info']['super_gen']['year_from']",
    "name"                : "['vehicle_info']['tech_param']['human_name']",
    "numberOfDoors"       : "['vehicle_info']['configuration']['doors_count']",
    "productionDate"      : "['documents']['year']",
    "vehicleTransmission" : "['vehicle_info']['tech_param']['transmission']",
    "engineDisplacement"  : "['vehicle_info']['tech_param']['human_name'].split()[0]",
    "enginePower"         : "['vehicle_info']['tech_param']['power']",
    "description"         : "['description']",
    "vendor"              : "['vehicle_info']['vendor']",
    "mileage"             : "['state']['mileage']",
    "Комплектация"        : "['vehicle_info']['complectation']",
    "Привод"              : "['lk_summary'].split(', ')[-2]",
    "Руль"                : "['vehicle_info']['steering_wheel']",
    "Владельцы"           : "['documents']['owners_number']",
    "ПТС"                 : "['documents']['pts']",
    "Таможня"             : "['documents']['custom_cleared']",
    "Владение"            : "['documents']['purchase_date']",
    "Price"               : "['price_info']['RUR']"
}

# Время паузы при ошибках от сервера (в сек.)
# Time delay when server request error occurs
SLEEP_TIME = 20

# Искомые марки автомобилей
# Car brands to look for
MARKS = ['MERCEDES', 'INFINITI', 'NISSAN', 'BMW', 'VOLKSWAGEN', 'MITSUBISHI', 
         'TOYOTA', 'VOLVO', 'SKODA', 'LEXUS', 'AUDI', 'HONDA', 'SUZUKI', 
         'HYUNDAI', 'KIA', 'CHEVROLET', 'SKODA', 'RENAULT', 'MITSUBISHI']

# Глобальная переменная со списком всех нужных марок и моделей для скрэпера
# Global varibale with list of all the car brands and models to give to the scraper
models_list = []

# Глобальная переменная для полученных результатов
# Global variable for obtained results
results = {}

In [27]:

def make_models_list():
    r = requests.get(URL_CATALOG)
    r.encoding = 'utf-8'
    r = r.text
    
    soup = BeautifulSoup(r, 'html.parser')
    rows = soup.find('div', class_ = 'sitemap').find_all('a')
    
    models_list = []
    for r in rows:
        parts = r.get('href').split('/')
        mark, model = parts[3].upper(), parts[4].upper()
        if mark in MARKS:
            models_list.append((mark, model))
    return models_list

In [28]:
def get_cars(m):
    for page in range(1, 100):
        PARAMS.update({
            'catalog_filter': [{"mark": m[0], "model": m[1]}],
            'page': page
        })
        
        while True:
            try:
                r = requests.post(URL, json = PARAMS, headers = HEADERS)
                break
            except:
                print(f'ERROR! Waiting {SLEEP_TIME} secs...')
                sleep(SLEEP_TIME)
                
        offers = r.json()['offers']

        if offers == []: break
        for i in offers:
            id_ = len(results)
            results[id_] = {}

            for k in CAR_PATTERN:
                try: v = eval('i' + CAR_PATTERN[k])
                except: v = None
                results[id_].update({k: v})
                
        print(f'id: [{id_}] page: [{page}] :: {m[0]} {m[1]}')

In [29]:
from multiprocessing.dummy import Pool as ThreadPool

def PerformScraping(this_day=False, multithreaded=False):
    global results
    results = {}
    models_list = make_models_list()
    PARAMS.update({ "top_days": int(this_day) })
    
    if multithreaded:
        pool = ThreadPool()
        pool.map(get_cars, models_list)
        pool.close()
        pool.join()
    else:
        for mm in models_list:
            get_cars(mm)

In [ ]:

'''
::: INSTRUCTIONS FOR USE :::

PerformScraping() function implements the entire web scraping 
process filling the 'results' dictionary with obtained data.
First, it generates the cars models list via make_models_list().
Then goes to scraping using get_cars().

The available param arguments are as follows:
    * this_day (default: False) 
        - defines whether the data will be searched for current day only;
          (this is used in order to make daily updates of the dataset);
    * multithreaded (default: False) 
        - toggles use of multithreading for faster experience; if neither defined, 
          scraper performs a standart one-threaded sequential processing
          (in case multithreading goes wrong).

Just uncomment the desired run mode:
'''

PerformScraping(this_day=False, multithreaded=True)
#PerformScraping(this_day=True, multithreaded=True)
# PerformScraping(this_day=False, multithreaded=False)
# PerformScraping(this_day=True, multithreaded=False)

id: [4] page: [1] :: AUDI 100
id: [5] page: [1] :: HONDA CAPA
id: [10] page: [1] :: HONDA MOBILIO
id: [15] page: [1] :: AUDI RS4
id: [16] page: [1] :: CHEVROLET CORSICA
id: [17] page: [1] :: CHEVROLET S_10
id: [54] page: [1] :: BMW X6
id: [59] page: [1] :: BMW 321
id: [96] page: [2] :: AUDI 100
id: [97] page: [1] :: HONDA CITY
id: [100] page: [1] :: HONDA MOBILIO_SPIKE
id: [107] page: [1] :: AUDI RS5
id: [144] page: [3] :: AUDI 100
id: [181] page: [2] :: BMW X6
id: [193] page: [1] :: CHEVROLET CORVETTE
id: [195] page: [1] :: CHEVROLET SILVERADO
id: [201] page: [1] :: HONDA N_BOX
id: [238] page: [4] :: AUDI 100
id: [248] page: [1] :: AUDI RS6
id: [285] page: [3] :: BMW X6
id: [291] page: [1] :: BMW 340
id: [328] page: [1] :: HONDA CIVIC
id: [329] page: [1] :: HONDA N_BOX_SLASH
id: [366] page: [5] :: AUDI 100
id: [403] page: [1] :: CHEVROLET CRUZE
id: [409] page: [1] :: AUDI RS7
id: [446] page: [2] :: HONDA CIVIC
id: [483] page: [4] :: BMW X6
id: [520] page: [1] :: CHEVROLET SPARK
id: [5

id: [5781] page: [1] :: BMW 2ER
id: [5818] page: [5] :: HYUNDAI ACCENT
id: [5855] page: [10] :: AUDI A4
id: [5892] page: [8] :: HONDA ACCORD
id: [5929] page: [4] :: CHEVROLET AVEO
id: [5966] page: [6] :: CHEVROLET LACETTI
id: [6003] page: [25] :: BMW 5ER
id: [6040] page: [11] :: AUDI A4
id: [6046] page: [1] :: BMW 2ACTIVETOURER
id: [6071] page: [1] :: HONDA FIT
id: [6108] page: [6] :: HYUNDAI ACCENT
id: [6126] page: [9] :: HONDA ACCORD
id: [6163] page: [26] :: BMW 5ER
id: [6200] page: [5] :: CHEVROLET AVEO
id: [6237] page: [7] :: CHEVROLET LACETTI
id: [6274] page: [12] :: AUDI A4
id: [6285] page: [1] :: BMW 2GRANDTOURER
id: [6322] page: [27] :: BMW 5ER
id: [6359] page: [7] :: HYUNDAI ACCENT
id: [6396] page: [8] :: CHEVROLET LACETTI
id: [6433] page: [6] :: CHEVROLET AVEO
id: [6436] page: [1] :: HONDA ACTY
id: [6473] page: [13] :: AUDI A4
id: [6510] page: [8] :: HYUNDAI ACCENT
id: [6512] page: [1] :: HONDA AIRWAVE
id: [6549] page: [1] :: BMW 2000_C_CS
id: [6586] page: [28] :: BMW 5ER
id:

id: [12595] page: [7] :: CHEVROLET NIVA
id: [12632] page: [11] :: BMW 7ER
id: [12669] page: [10] :: HYUNDAI SOLARIS
id: [12706] page: [28] :: BMW 3ER
id: [12743] page: [12] :: HYUNDAI ELANTRA
id: [12747] page: [12] :: BMW 7ER
id: [12784] page: [13] :: AUDI A6
id: [12821] page: [2] :: INFINITI QX70
id: [12841] page: [29] :: BMW 3ER
id: [12878] page: [8] :: CHEVROLET NIVA
id: [12915] page: [11] :: HYUNDAI SOLARISid: [12918] page: [13] :: HYUNDAI ELANTRA

id: [12955] page: [1] :: KIA OPTIMA
id: [12992] page: [14] :: AUDI A6
id: [13017] page: [3] :: INFINITI QX70
id: [13054] page: [14] :: HYUNDAI ELANTRA
id: [13091] page: [9] :: CHEVROLET NIVA
id: [13128] page: [12] :: HYUNDAI SOLARIS
id: [13144] page: [1] :: BMW 8ER
id: [13181] page: [2] :: KIA OPTIMA
id: [13218] page: [15] :: AUDI A6
id: [13255] page: [10] :: CHEVROLET NIVA
id: [13292] page: [13] :: HYUNDAI SOLARIS
id: [13329] page: [16] :: AUDI A6
id: [13366] page: [1] :: INFINITI QX80
id: [13403] page: [11] :: CHEVROLET NIVA
id: [13440

id: [19468] page: [7] :: LEXUS RX
id: [19505] page: [1] :: KIA RIO
id: [19542] page: [9] :: HYUNDAI TUCSON
id: [19567] page: [5] :: MERCEDES G_KLASSE
id: [19628] page: [14] :: KIA CEEDid: [19641] page: [5] :: BMW X3

id: [19678] page: [8] :: LEXUS RX
id: [19679] page: [1] :: AUDI Q2
id: [19716] page: [10] :: HYUNDAI TUCSON
id: [19718] page: [11] :: HYUNDAI TUCSON
id: [19755] page: [1] :: HYUNDAI I30
id: [19792] page: [15] :: KIA CEED
id: [19829] page: [6] :: BMW X3
id: [19866] page: [2] :: KIA RIO
id: [19903] page: [9] :: LEXUS RX
id: [19940] page: [1] :: MERCEDES G_KLASSE_AMG
id: [19977] page: [1] :: AUDI Q3
id: [20014] page: [16] :: KIA CEED
id: [20017] page: [1] :: HYUNDAI TUSCANI
id: [20054] page: [7] :: BMW X3
id: [20091] page: [10] :: LEXUS RX
id: [20128] page: [2] :: HYUNDAI I30
id: [20165] page: [2] :: MERCEDES G_KLASSE_AMG
id: [20202] page: [3] :: KIA RIO
id: [20239] page: [17] :: KIA CEED
id: [20276] page: [2] :: AUDI Q3
id: [20313] page: [8] :: BMW X3
id: [20322] page: [1] :

id: [26506] page: [13] :: BMW X5
id: [26543] page: [6] :: MERCEDES C_KLASSE
id: [26574] page: [1] :: AUDI Q8
id: [26611] page: [32] :: KIA RIO
id: [26634] page: [1] :: HYUNDAI IX55
id: [26671] page: [4] :: MERCEDES GLC_KLASSE
id: [26708] page: [14] :: BMW X5
id: [26745] page: [7] :: MERCEDES C_KLASSE
id: [26747] page: [1] :: MERCEDES SLK_KLASSE_AMG
id: [26755] page: [5] :: MERCEDES GLC_KLASSE
id: [26756] page: [1] :: AUDI R8
id: [26793] page: [15] :: BMW X5
id: [26830] page: [33] :: KIA RIO
id: [26832] page: [1] :: KIA FORTE
id: [26833] page: [1] :: MERCEDES SLR_KLASSE
id: [26837] page: [1] :: MERCEDES GLC_KLASSE_AMG
id: [26874] page: [34] :: KIA RIO
id: [26911] page: [16] :: BMW X5
id: [26924] page: [1] :: HYUNDAI LANTRA
id: [26961] page: [8] :: MERCEDES C_KLASSE
id: [26966] page: [1] :: MERCEDES SLS_AMG
id: [26970] page: [1] :: AUDI RS3
id: [27007] page: [35] :: KIA RIO
id: [27008] page: [1] :: KIA JOICE
id: [27045] page: [9] :: MERCEDES C_KLASSE
id: [27082] page: [17] :: BMW X5
id: 

...

id: [37828] page: [17] :: MITSUBISHI OUTLANDER
id: [37865] page: [7] :: MERCEDES S_KLASSE
id: [37866] page: [1] :: NISSAN PRIMASTAR
id: [37875] page: [1] :: NISSAN ELGRAND
id: [37912] page: [18] :: MITSUBISHI OUTLANDER
id: [37949] page: [2] :: NISSAN MURANO
id: [37986] page: [5] :: NISSAN X_TRAIL
id: [38023] page: [6] :: RENAULT DUSTER
id: [38060] page: [8] :: MERCEDES S_KLASSE
id: [38089] page: [19] :: MITSUBISHI OUTLANDER
id: [38126] page: [1] :: KIA SPORTAGE
id: [38163] page: [7] :: RENAULT DUSTER
id: [38200] page: [3] :: NISSAN MURANO
id: [38237] page: [1] :: NISSAN PRIMERA
id: [38274] page: [6] :: NISSAN X_TRAIL
id: [38311] page: [9] :: MERCEDES S_KLASSE
id: [38348] page: [2] :: KIA SPORTAGE
id: [38385] page: [4] :: NISSAN MURANO
id: [38422] page: [8] :: RENAULT DUSTER
id: [38459] page: [1] :: SKODA CITIGO
id: [38496] page: [7] :: NISSAN X_TRAIL
id: [38533] page: [2] :: NISSAN PRIMERA
id: [38570] page: [10] :: MERCEDES S_KLASSE
id: [38607] page: [1] :: MITSUBISHI PAJERO
id: [38644

id: [44164] page: [28] :: KIA SPORTAGE
id: [44201] page: [1] :: RENAULT KAPTUR
id: [44238] page: [4] :: SKODA OCTAVIA
id: [44239] page: [1] :: TOYOTA AVANZA
id: [44242] page: [1] :: SUZUKI HUSTLER
id: [44279] page: [1] :: MITSUBISHI SAVRIN
id: [44316] page: [29] :: KIA SPORTAGE
id: [44353] page: [17] :: NISSAN QASHQAI
id: [44390] page: [5] :: SKODA OCTAVIA
id: [44427] page: [1] :: TOYOTA AVENSIS
id: [44453] page: [1] :: SUZUKI IGNIS
id: [44464] page: [30] :: KIA SPORTAGE
id: [44501] page: [18] :: NISSAN QASHQAI
id: [44538] page: [1] :: RENAULT KOLEOS
id: [44540] page: [1] :: MITSUBISHI SIGMA
id: [44577] page: [1] :: RENAULT CAPTUR
id: [44614] page: [6] :: SKODA OCTAVIA
id: [44651] page: [2] :: TOYOTA AVENSIS
id: [44682] page: [2] :: RENAULT KOLEOS
id: [44719] page: [19] :: NISSAN QASHQAI
id: [44756] page: [1] :: SUZUKI JIMNY
id: [44793] page: [3] :: TOYOTA AVENSIS
id: [44800] page: [1] :: MITSUBISHI SPACE_GEAR
id: [44837] page: [1] :: KIA STINGER
id: [44874] page: [20] :: NISSAN QASHQA

id: [49751] page: [1] :: TOYOTA HARRIER
id: [49758] page: [1] :: SUZUKI WAGON_R_PLUS
id: [49795] page: [24] :: RENAULT LOGAN
id: [49832] page: [37] :: SKODA OCTAVIA
id: [49833] page: [1] :: TOYOTA PIXIS_VAN
id: [49870] page: [18] :: TOYOTA CAMRY
id: [49907] page: [5] :: TOYOTA RAV_4
id: [49944] page: [7] :: NISSAN TEANA
id: [49981] page: [38] :: SKODA OCTAVIA
id: [50018] page: [25] :: RENAULT LOGAN
id: [50055] page: [19] :: TOYOTA CAMRYid: [50056] page: [1] :: SUZUKI X_90

id: [50093] page: [1] :: TOYOTA HIACE
id: [50130] page: [39] :: SKODA OCTAVIA
id: [50167] page: [8] :: NISSAN TEANA
id: [50204] page: [6] :: TOYOTA RAV_4
id: [50214] page: [1] :: TOYOTA PLATZ
id: [50251] page: [26] :: RENAULT LOGAN
id: [50264] page: [2] :: TOYOTA HIACE
id: [50301] page: [40] :: SKODA OCTAVIA
id: [50338] page: [20] :: TOYOTA CAMRY
id: [50375] page: [9] :: NISSAN TEANA
id: [50412] page: [7] :: TOYOTA RAV_4
id: [50449] page: [27] :: RENAULT LOGAN
id: [50452] page: [1] :: TOYOTA PORTE
id: [50489] page: [

id: [55279] page: [12] :: TOYOTA LAND_CRUISER
id: [55284] page: [1] :: VOLVO 440
id: [55321] page: [5] :: SKODA SUPERB
id: [55322] page: [1] :: TOYOTA SPRINTER_TRUENO
id: [55359] page: [8] :: TOYOTA COROLLA
id: [55396] page: [5] :: VOLKSWAGEN PASSAT
id: [55433] page: [13] :: TOYOTA LAND_CRUISER
id: [55470] page: [11] :: RENAULT SANDERO
id: [55531] page: [1] :: VOLKSWAGEN ARTEONid: [55544] page: [9] :: TOYOTA COROLLA

id: [55581] page: [6] :: SKODA SUPERB
id: [55618] page: [6] :: VOLKSWAGEN PASSAT
id: [55626] page: [1] :: TOYOTA STARLET
id: [55634] page: [1] :: VOLVO 460
id: [55671] page: [14] :: TOYOTA LAND_CRUISER
id: [55708] page: [7] :: VOLKSWAGEN PASSAT
id: [55745] page: [7] :: SKODA SUPERB
id: [55747] page: [1] :: VOLKSWAGEN ATLAS
id: [55784] page: [12] :: RENAULT SANDERO
id: [55786] page: [1] :: TOYOTA SUCCEED
id: [55788] page: [1] :: TOYOTA COROLLA_II
id: [55825] page: [15] :: TOYOTA LAND_CRUISER
id: [55841] page: [8] :: SKODA SUPERB
id: [55878] page: [8] :: VOLKSWAGEN PASSAT
id

id: [60586] page: [2] :: VOLVO XC90
id: [60623] page: [37] :: VOLKSWAGEN PASSAT
id: [60643] page: [5] :: VOLVO S80
id: [60680] page: [9] :: VOLKSWAGEN GOLF
id: [60717] page: [3] :: VOLVO XC90
id: [60754] page: [38] :: VOLKSWAGEN PASSAT
id: [60791] page: [10] :: VOLKSWAGEN GOLF
id: [60817] page: [1] :: VOLVO S90
id: [60854] page: [39] :: VOLKSWAGEN PASSAT
id: [60891] page: [4] :: VOLVO XC90
id: [60928] page: [11] :: VOLKSWAGEN GOLF
id: [60965] page: [5] :: VOLVO XC90
id: [61002] page: [40] :: VOLKSWAGEN PASSAT
id: [61019] page: [1] :: VOLVO V40
id: [61036] page: [41] :: VOLKSWAGEN PASSAT
id: [61073] page: [12] :: VOLKSWAGEN GOLF
id: [61110] page: [6] :: VOLVO XC90
id: [61147] page: [13] :: VOLKSWAGEN GOLF
id: [61184] page: [7] :: VOLVO XC90
id: [61218] page: [1] :: VOLVO V40_CC
id: [61220] page: [1] :: VOLKSWAGEN PASSAT_NA
id: [61257] page: [14] :: VOLKSWAGEN GOLF
id: [61293] page: [8] :: VOLVO XC90
id: [61330] page: [15] :: VOLKSWAGEN GOLF
id: [61335] page: [1] :: VOLVO V50
id: [61372]

In [38]:
# Сохраняем полученные данные
# Save the obtained data

from datetime import datetime
now = datetime.now().strftime('[%d.%m.%Y]-[%H-%M]')
prefix = 'auto_ru_Scraped'
filename = f'{prefix}-{now}.csv'

from pandas import DataFrame
DataFrame(results).T.to_csv(filename, index = False)
print('Written file:', filename)

Written file: auto_ru_Scraped-[26.05.2021]-[14-08].csv


In [39]:
df = pd.read_csv('auto_ru_Scraped-[26.05.2021]-[14-08].csv')

In [40]:
pd.set_option('display.max_rows', 50)  
pd.set_option('display.max_columns', 50)

In [41]:
df.sample(6)

,bodyType,brand,model_name,color,fuelType,modelDate,name,numberOfDoors,productionDate,vehicleTransmission,engineDisplacement,enginePower,description,vendor,mileage,Комплектация,Привод,Руль,Владельцы,ПТС,Таможня,Владение,Price
8120,Седан,BMW,3ER,0000CC,бензин,2015.0,318i 1.5 AT (136 л.с.),4.0,2016,AUTOMATIC,318i,136.0,Автомобиль продается в кредит.\n\nАуди Центр С...,EUROPEAN,95185,{'id': '0'},задний,LEFT,1.0,ORIGINAL,True,NaN,1549000
53669,Внедорожник 5 дв.,TOYOTA,LAND_CRUISER,CACECB,бензин,2012.0,4.6 AT (309 л.с.) 4WD,5.0,2012,AUTOMATIC,4.6,309.0,"На автомобиль установлено ГБО в ПТС вписано, в...",JAPANESE,211876,{'id': '0'},полный,LEFT,2.0,ORIGINAL,True,NaN,2350000
30832,Внедорожник 5 дв.,KIA,SELTOS,0000CC,бензин,2019.0,2.0 CVT (149 л.с.),5.0,2020,VARIATOR,2.0,149.0,"Продаю Киа селтос 2020 Style, без дтп, на гара...",KOREAN,8500,"{'id': '21785758', 'name': 'Style', 'available...",передний,LEFT,1.0,ORIGINAL,True,"{'year': 2020, 'month': 6}",1820000
40913,Хэтчбек 5 дв.,SKODA,FABIA,040001,бензин,2010.0,1.2 MT (60 л.с.),5.0,2010,MECHANICAL,1.2,60.0,экономичный автомобиль для города!ездила жена ...,EUROPEAN,137000,{'id': '0'},передний,LEFT,3.0,ORIGINAL,True,NaN,310000
60141,Универсал 5 дв.,VOLVO,XC70,200204,дизель,2013.0,2.4d AT (215 л.с.) 4WD,5.0,2014,AUTOMATIC,2.4d,215.0,Продается автомобиль Volvo XC70 в идеальном со...,EUROPEAN,268220,{'id': '0'},полный,LEFT,1.0,ORIGINAL,True,NaN,1580000
48358,Хэтчбек 5 дв.,SUZUKI,SX4,CACECB,бензин,2006.0,1.6 MT (106 л.с.),5.0,2007,MECHANICAL,1.6,106.0,Нюансы по кузову. Подробности по телефону. Ход...,JAPANESE,154000,{'id': '0'},передний,LEFT,4.0,DUPLICATE,True,"{'year': 2019, 'month': 6}",395000


In [42]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65127 entries, 0 to 65126
Data columns (total 23 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   bodyType             65126 non-null  object 
 1   brand                65127 non-null  object 
 2   model_name           65127 non-null  object 
 3   color                65127 non-null  object 
 4   fuelType             65127 non-null  object 
 5   modelDate            65126 non-null  float64
 6   name                 65126 non-null  object 
 7   numberOfDoors        65126 non-null  float64
 8   productionDate       65127 non-null  int64  
 9   vehicleTransmission  65126 non-null  object 
 10  engineDisplacement   65126 non-null  object 
 11  enginePower          65126 non-null  float64
 12  description          63095 non-null  object 
 13  vendor               65127 non-null  object 
 14  mileage              65127 non-null  int64  
 15  Комплектация         65127 non-null 